# Main: 

This notebook, as long as it's run from the hdfs repo of one of our users (as set in the beginning of the notebook) should be enough to obtain the desired planner as all dataframes necessary are stored on our repos. Otherwise, you are welcome to first run the other notebooks and run from your repo.

In [ ]:
%%configure
{"pyFiles": ["/user/gottraux/dijkstra_algorithms.py"],
 "conf": {
    "spark.app.name": "dslab-group_final"
}}

### Imports and helper functions:

In [ ]:
import pickle
import json
import time
import networkx as nx
import pandas as pd
from pyspark.sql.functions import col

MAX_TRIP_DURATION = 2 #duration in hour 

days_dict = {0: 'monday', 1: 'tuesday', 2: 'wednesday', 3: 'thursday', 4: 'friday'}

Set which home will be used to load the data

In [ ]:
%%local
import os
import pandas as pd
username = 'liseli'

In [ ]:
%%send_to_spark -i username -t str -n username

## Load graph data: 

Load information to create our network. 

In [ ]:
trips = spark.read.format('orc').load('/data/sbb/timetables/orc/trips/000000_0')
calendar = spark.read.format('orc').load('/data/sbb/timetables/orc/calendar/000000_0')

nodes_df = spark.read.orc("/user/{}/nodes.orc".format(username))
edges_df = spark.read.orc("/user/{}/edges_with_mean_and_std_sec.orc".format(username))

nodes = nodes_df.rdd.map(lambda r: (r[0], {'name': r['stop_name'],
                                              'lat': r['stop_lat'],
                                              'lon': r['stop_lon']})).collect()
# reverse edges by:
edges_df = edges_df.withColumnRenamed('stop_id', 'temp')\
                    .withColumnRenamed('next_stop', 'stop_id').withColumnRenamed('temp', 'next_stop')

Load walking edges from pickle file:

In [ ]:
%%local
walking_times = pd.read_pickle('walking_edges.pickle')

In [ ]:
%send_to_spark -i walking_times -t df -m 20000

In [ ]:
# reverse edges for walking edges also:
walking_times = (walking_times.withColumnRenamed('source', 'temp').withColumnRenamed('target', 'source').withColumnRenamed('temp', 'target'))

edges_walking = walking_times.toPandas()
edges_walking['attrs'] = edges_walking.apply(lambda x: {'time': -1, 'duration': x['walk_duration']}, axis=1)
edges_walking = list(edges_walking[['source', 'target', 'attrs']].to_numpy())

## Functions for the creation of the graph

In [ ]:
def day_trips(*day_ids):
    """
    day_trips: gives the trip_ids that operate on certain days
    input: a variable number of day ids
    output:s spark dataframe with trip_ids
    
    """
    days = [days_dict[day_id] for day_id in day_ids]
    where_clause = " and ".join(days)

    day_services = calendar.where(where_clause).select('service_id')
    return day_services.join(trips, on='service_id').select('trip_id')

def create_edges_for_trip(edges_df, day_id, arrival_time):
    """
    create_edges_for_trip: constructs edges (and thus trips) that exist in a window of two hours before a given input time
    @input:
    - edges_df: df from which we construct the edges
    - day_id: id of week-day (e.g. wednesday is day id 2, see dictionnary above)
    - hour, minute: time at which we want to arrive somewhere (e.g. 11:30)
    @output: data frame of selected edges
    """
    #select only the trips that occur on that day:
    edges_df= edges_df.join(day_trips(day_id), on='trip_id')
    
    min_dep_time = arrival_time - 60*MAX_TRIP_DURATION
    
    #keep only those in a window of two hours:
    edges_df = edges_df.filter((col('departure_time') > min_dep_time) & 
                                            (col('arrival_time') <= arrival_time))
    
    edges = edges_df.rdd.map(lambda r: (r['stop_id'], r['next_stop'], {'duration': r['trip_duration'],
                                                                       'time': float(r['departure_time']),
                                                                       'trip_id': r['trip_id'],
                                                                       'mean': r['mean'],
                                                                       'std': r['std'],
                                                                       'transport': r['train_type'], 
                                                                      'p_90':r['p_90'],
                                                                       'p_91':r['p_91'],
                                                                       'p_92':r['p_92'],
                                                                       'p_93':r['p_93'],
                                                                       'p_94':r['p_94'],
                                                                       'p_95':r['p_95'],
                                                                       'p_96':r['p_96'],
                                                                       'p_97':r['p_97'],
                                                                       'p_98':r['p_98'],
                                                                       'p_99':r['p_99']})).collect()
    
    return edges + edges_walking

## Dijkstra's algorithm for shortest path:

See `README.md`for description of algorith. 

#### Validation for Dijkstra:
##### Feasible paths: 
`is_path_valid` is a function that looks through a path to see if it is valid. 
So it looks for:
- missed connections
- transfer time of less than 2 minutes between two transports

##### Validate a path:

`validate_path_`:for a given path from Dijkstra, this function samples delays for transfers where we go from a transport -> walk or transport -> transport. 
- For transport 1 -> transport 2: the delay of transport 1 will be added to its trip duration
- For transport -> walk: the delay of transport will be added to the departure time of walk 

After modifying these values, the function checks if the path is still feasible. This operation is repeated a certain number of times and if the percentage of feasible paths is higher or equal to the wished confidence, the path is validated. 

This function is called in Dijkstra. As we wish a path validated for a certain confidence, if Dijkstra's path is not validated, Dijkstra will increase its confidence till it gets a valid path or none.

To not use too much space in this notebook, they are in a dedicated file (with all functions related to Dijsktra). Default to /user/gottraux/dijkstra_algorithms.py on the hdfs filesystem (loaded in the spark context at the top of the notebook).

In [ ]:
"""
To load (or reload) into hdfs, need to update cell at the top of the notebook and restart kernel

hdfs dfs -rm /user/${JUPYTERHUB_USER}/dijkstra_algorithms.py 2>/dev/null
hdfs dfs -copyFromLocal notebooks/dijkstra_algorithms.py /user/${JUPYTERHUB_USER}/
"""
from dijkstra_algorithms import *

## Visualisation:

In [ ]:
%%local
import ipywidgets as widgets
import fuzzy_pandas as fpd
import time

def search_station(station):
    search = pd.DataFrame([station], columns=['station'])
    matches = fpd.fuzzy_merge(search, stations_name, left_on='station', right_on='stop_name',
                              ignore_case=True, ignore_nonalpha=True, ignore_nonlatin=True, ignore_order_words=True,
                              keep='match', threshold=0.8, method='jaro')
    return matches['stop_name'].to_list()

def search_station_departure(sender):
    phrase = depart_station.value
    depart_proposals.options = search_station(phrase)
    
def search_station_arrival(sender):
    phrase = arrive_station.value
    arrive_proposals.options = search_station(phrase)
    
no_station_selected = "None selected"

def select_station_departure(sender):
    if(sender['name'] == 'label'):
        if(sender['new'] == None):
            selected_depart_station.value = no_station_selected
        else:
            selected_depart_station.value = sender['new']
            
def select_station_arrival(sender):
    if(sender['name'] == 'label'):
        if(sender['new'] == None):
            selected_arrival_station.value = no_station_selected
        else:
            selected_arrival_station.value = sender['new']
            
def find_route_button(button):
    # Parse arguments
    depart_station_str = selected_depart_station.value
    if depart_station_str == no_station_selected:
        report_error("No departure station selected")
        return
    
    arrive_station_str = selected_arrival_station.value
    if arrive_station_str == no_station_selected:
        report_error("No arrival station selected")
        return
    
    date = date_picker.value
    if(date == None):
        report_error("No date selected")
        return
    
    if(date.weekday() > 4):
        report_error("Date is a weekend day, please select a week day")
        return
    
    hour_str = hour_picker.value
    
    if hour_str == None or hour_str == "":
        report_error("No hour selected")
        return
    
    hour_str = hour_str.split(':')
    hour = -1
    minute = -1
    
    try:
        if(len(hour_str) != 2):
            raise Error
        hour = int(hour_str[0])
        minute = int(hour_str[1])        
    except:
        report_error("Invalid hour format, use HH:MM")
        return
    
    if(hour not in range(8,21)):
        report_error("Invalid hour, valid range: [8,20]")
        return
            
    if(minute not in range(0,60)):
        report_error("Invalid minute, valid range: [0,59]")
        return
        
    confidence = confidence_picker.value
        
    report_error(None)
    
    
    # Show progress bar
    results.children = []
    progress_bar.layout = widgets.Layout(display='block')
    
    # Send variables to spark
    # Convert to str for 'send_to_spark'
    date_str = date.strftime("%Y-%m-%d")
    hour_str = str(hour)
    minute_str = str(minute)
    confidence_str = str(confidence)
    get_ipython().push(['depart_station_str', 'arrive_station_str', 'date_str', 'hour_str', 'minute_str', 'confidence_str'])
    get_ipython().run_cell_magic('send_to_spark', ' -i depart_station_str -t str -n depart_station_str', ' ')
    get_ipython().run_cell_magic('send_to_spark', ' -i arrive_station_str -t str -n arrive_station_str', ' ')
    get_ipython().run_cell_magic('send_to_spark', ' -i date_str -t str -n date_str', ' ')
    get_ipython().run_cell_magic('send_to_spark', ' -i hour_str -t str -n hour_str', ' ')
    get_ipython().run_cell_magic('send_to_spark', ' -i minute_str -t str -n minute_str', ' ')
    get_ipython().run_cell_magic('send_to_spark', ' -i confidence_str -t str -n confidence_str', ' ')

    # Run algorithm
    get_ipython().run_cell_magic('spark', '', """
    result = get_result_path(depart_station_str, arrive_station_str, date_str, hour_str, minute_str, confidence_str)
    """)
    
    # Retrive results from spark
    get_ipython().run_cell_magic('spark', '-o result', ' ')
    
    # Display path
    progress_bar.layout = widgets.Layout(display='none')
    display_path(result)
    
def display_path(path):
    stops = []
    
    for index, row in path.iterrows():
        start = row['from']
        end = row['to']
        duration = row['duration']
        departure_time = row['departure_time'].strftime("%H:%M")
        walk = row['walk']
        transport = row['transport']
        
        message =  f"<p style='font-size: 20px; padding-bottom:10px;'>{transport}, {start} <b>&rarr;</b> {end}, {duration:.1f} minutes</p>" if walk\
                    else f"<p style='font-size: 20px; padding-bottom:10px;'>{transport}, {departure_time}: {start} <b>&rarr;</b> {end}, {duration:.1f} minutes</p>"
        
        stops.append(widgets.HTML(value=message))
        
    results.children = stops
    
def report_error(error_message):
    if error_message == None:
        error.value = ""
    else:
        error.value = "<b style='color:red;'>Error: " + error_message  + "</b>"

In [ ]:
def get_result_path(start_str, end_str, date_str, hour_str, minute_str, confidence_str):
    day_id = time.strptime(date_str, "%Y-%m-%d").tm_wday
    
    arrival_hour = int(hour_str)
    arrival_minute = int(minute_str)
    arrival_time = arrival_hour*60+arrival_minute
    departure_time = arrival_time - MAX_TRIP_DURATION*60
    
    confidence = int(confidence_str) / 100.0
    
    print("Parsed arguments")
    
    source = nodes_df.where(col('stop_name') == start_str).take(1)[0][0][:7]
    target = nodes_df.where(col('stop_name') == end_str).take(1)[0][0][:7]
    
    print("Found ids")
    
    # Put in function
    edges = create_edges_for_trip(edges_df, day_id, arrival_time)
    graph = nx.MultiDiGraph()
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)
    
    print("Created graph")

    old_number_of_nodes = graph.number_of_nodes()
    # Remove unreachable nodes
    dists, paths = normal_dijkstra(graph, '8503000')
    not_reachable = set(graph.nodes) - set(dists.keys())
    _ = graph.remove_nodes_from(list(not_reachable))
    print('{} nodes removed'.format(old_number_of_nodes - graph.number_of_nodes()))

    # Temp for problem of name's encoding
    import unicodedata
    nodes_data = graph.nodes(data=True)
    for n in graph.nodes:
        nodes_data[n]['name'] = unicodedata.normalize('NFKD', nodes_data[n]['name']).encode('ascii','ignore')
    
    
    print("Ran on cluster !")
    print("From {} to {} on {} at {}:{} for confidence {}".format(start_str, end_str, date_str, hour_str, minute_str, confidence_str))
    
    path = dijkstra_reversed(graph, source, arrival_time, last_target=target, confidence=confidence)
    return spark.createDataFrame(path)

stations_name = nodes_df.select('stop_name').distinct()

In [ ]:
%%spark -o stations_name

In [ ]:
%%local
style = {'description_width': 'initial'}

# Search station
depart_station = widgets.Text(description = 'Search departure station',
                              layout=widgets.Layout(width='40%'),
                              style=style)
depart_station.observe(search_station_departure)
arrive_station = widgets.Text(description = 'Search arrival station',
                              layout=widgets.Layout(width='40%'),
                              style=style)
arrive_station.observe(search_station_arrival)


# Proposals
depart_proposals = widgets.Select(description = 'Found stations',
                                  layout=widgets.Layout(width='40%', height='200px'),
                                  style=style)
depart_proposals.observe(select_station_departure)
arrive_proposals = widgets.Select(description = 'Found stations',
                                  layout=widgets.Layout(width='40%', height='200px'),
                                  style=style)
arrive_proposals.observe(select_station_arrival)


# Stations
selected_depart_station = widgets.Label(value = no_station_selected, style=style)
selected_box_depart_station = widgets.HBox([widgets.Label(value = "Selected depart station: ", style=style),
                                             selected_depart_station], layout=widgets.Layout(width='40%'))
selected_arrival_station = widgets.Label(value = no_station_selected, style=style)
selected_box_arrival_station = widgets.HBox([widgets.Label(value = "Selected arrival station: ", style=style),
                                             selected_arrival_station], layout=widgets.Layout(width='40%'))



# Options
date_picker = widgets.DatePicker(
                    description='Pick a Date',
                    disabled=False,
                    layout=widgets.Layout(width='20%')
                )
hour_picker = widgets.Text(description = 'Arrival time',
                            placeholder='HH:MM',
                            layout=widgets.Layout(width='20%'),
                            style=style
                          )
confidence_picker = widgets.IntSlider(
            value=95,
            min=90,
            max=99,
            step=1,
            description='Confidence:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            layout=widgets.Layout(width='25%'),
            style=style
        )
search_button = widgets.Button(
            description='Find route',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Find route',
            icon='check', # (FontAwesome names without the `fa-` prefix)
            layout=widgets.Layout(width='15%')
        )
search_button.on_click(find_route_button)


# Error
error = widgets.HTML(value="")


padding = widgets.HTML(value="", layout=widgets.Layout(height='50px'))

# Progress bar
progress_bar = widgets.HTML(value="Finding best route...", layout=widgets.Layout(display='none'))

# Result
results = widgets.VBox([])

stations = widgets.HBox([depart_station, arrive_station])
proposals = widgets.HBox([depart_proposals, arrive_proposals])
selected_stations = widgets.HBox([selected_box_depart_station, selected_box_arrival_station])
options = widgets.HBox([date_picker, hour_picker, confidence_picker, search_button])
layout = widgets.VBox([stations, proposals, selected_stations, options, error, padding, progress_bar, results])

layout

**Note**: for visualisation most stations start with `Zürich, ...`. Try with that if you can't find your station.

## Run the algorithm without the visualisation

Choose an arrival time to create a graph: (all edges will leave at most 2 hours before this arrival time)

In [ ]:
day_id, arrival_hour, arrival_minute = 4, 12, 30

Create the network

In [ ]:
edges = create_edges_for_trip(edges_df, day_id, arrival_hour*60+arrival_minute)

graph = nx.MultiDiGraph()
graph.add_nodes_from(nodes)
graph.add_edges_from(edges)

old_number_of_nodes = graph.number_of_nodes()
# Remove unreachable nodes (e.g. those that have a trip outside of 15km to reach them)
dists, paths = normal_dijkstra(graph, '8503000')
not_reachable = set(graph.nodes) - set(dists.keys())
_ = graph.remove_nodes_from(list(not_reachable))

# Temp for problem of name's encoding:
import unicodedata
nodes_data = graph.nodes(data=True)
for n in graph.nodes:
    nodes_data[n]['name'] = unicodedata.normalize('NFKD', nodes_data[n]['name']).encode('ascii','ignore')

### Test Dijkstra's reversed algorithm: 

In [ ]:
# Tao's example: 
"""
Route 1: HB -> Auszelg
20.TA.26-9-A-j19-1.2.H: 8503000:0:41/42 at 12:07:00 ~ 8503310:0:3 at 12:17:00
Walking: 8503310:0:3 ~ 8590620
168.TA.26-12-A-j19-1.2.H: 8590620 at 12:23:00 ~ 8591049 at 12:29:00
"""
best_path1 = dijkstra_reversed(graph, '8503000', arrival_hour*60+arrival_minute, last_target='8591049', confidence=0.99)

In [ ]:
# Another example, with and without confidence: 
"""
Route 2:
Zurich, Triemli (8503610) to Zurich Altstetten, Bahnhof N (8591057)
"""
# From Triemli to Altstetten bahnhof:
print('Without confidence ->')
best_path1 = dijkstra_reversed(graph, '8503610', arrival_hour*60+arrival_minute, last_target='8591057')
print('')
print('With confidence of 98% ->')
best_path1 = dijkstra_reversed(graph, '8503610', arrival_hour*60+arrival_minute, last_target='8591057', confidence=0.98)